Imports

In [1]:
import pandas as pd
from ast import literal_eval
import numpy as np
import json

TF-IDF class. Contains methods which can easily access relevant information from the csv file which the class reads, which represents the TF-IDF model.

TF-IDF stands for Term Frequency - Inverse Document Frequency. It gives each keyword-document combination a weight, based on how often it occurs in the document, but also based on how often it occurs in all documents. This ensures that words which occur often in many documents do not receive high weights, rather only words which are relatively unique to a document and occur frequently in that document get high weights.

If all documents are about topic A, we do not care that document x contains is about document A.

Implementation wise, the TF-IDF is a csv file with each row representing a document and each column representing a keyword, the value at i,j represents the weight that keyword j has in document i.

In [4]:
class TfIdf:
    def __init__(self):
        # Loading in the tf-idf
        # Rows are documents, the first column is the document id
        # Columns are keywords, the first row is the keyword id
        self.tfidf = pd.read_csv("C:/Users/01din\Desktop\data APG/tfidf/tf_idf_new.csv") #your_filepath_here

        # Loading in the keywords
        # Two columns, column 1 is id, column 2 is keyword
        # We can access the weight in the tf-idf by first accessing the id number from the keyword file
        self.tfidf_keywords = pd.read_csv("C:/Users/01din\Desktop\data APG/tfidf/tf_idf_keywords_new.csv") #your_filepath_here
        self.tfidf_keywords.columns = ["id", "keyword"]

    def get_id_by_keyword(self, keyword):
        return self.tfidf_keywords.id.iloc[self.tfidf_keywords[self.tfidf_keywords.keyword == keyword].index].values[0]


    def get_keyword_by_id(self, id):
        return self.tfidf_keywords.loc[self.tfidf_keywords.id==id, 'keyword'].iloc[0]
    # Method to get the tfidf value of a keyword in a page
    def get_tf_idf_value(self, page_id, keyword):
        keyword_id = self.get_id_by_keyword(keyword)
        return self.tfidf.iloc[page_id-1, keyword_id + 1]

    def get_all_keywords_by_id(self, page_id):
        keywords = self.tfidf.iloc[page_id-1][1:].values
        return keywords

    def get_all_keywords_by_id_normalized(self, page_id):
        keywords_weights = self.get_all_keywords_by_id(page_id)
        sum_value = sum(keywords_weights)
        if sum_value == 0:
            return keywords_weights
        keywords_weights_normalized = keywords_weights/sum_value
        return keywords_weights_normalized

    def get_number_of_keywords(self):
        return self.tfidf_keywords.shape[0]

Instantiate the object

In [5]:
tfidf = TfIdf()

Read in the dataset of visits, used to test the information need prediction.

In [6]:
generic = lambda x: literal_eval(x)

conv = {'url_id_path': generic,
        'seconds_spent_path': generic}
df = pd.read_csv('C:/Users/01din\Desktop\data APG\keywords_split/visits_dino.csv', converters=conv) #your_filepath_here
df_2 = pd.read_csv('C:/Users/01din\Desktop\data APG\keywords_split/visits_matthijs.csv', converters=conv) #your_filepath_here

Create seperate dataframes for the paths taken and for the seconds spent on each url.
In paths, index 0 gives a list of urls, the equivalent index 0 in seconds represents how many seconds the user spent on the equal index urls.

In [8]:
visit_ids = df['visitor_id[visit_number]']
paths = df.url_id_path
visit_ids_2 = df_2['visitor_id[visit_number]']
paths_2 = df_2.url_id_path
#secs_spent = df.seconds_spent_path NOT USING TIME SO I GUESS THIS IS UNNECESSARY
print(len(paths)) # SHOULD BE 360511 FOR DINO, LAURENCE, AND I - BUT 360513 FOR MATTHIJS
print(len(paths_2))

360511
360513


In [9]:
# NOT SURE WHAT THIS IS FOR
#list = tfidf.get_all_keywords_by_id(1)
#for i in range(0, len(list)):
#    if list[i]!=0:
#        print(i)

)Simplified implementation of IUNIS, which simply looks at the keywords which the user comes across, does not use the adjacency matrix, proximal cues, and spreading activation.
It seems that these are mostly important for WUFIS, which we are not interested in, thus implementing it is not worth the effort.

Input is the ordered list of page id's that the user has visited.
For each page, take the TF-IDF values from that page, which is a row of keywords, with either 0 if the keyword is not present or [0,1] if it is.

Then, for each next page, we add the TF-IDF values onto the existing weights.

Eventually we can sort the list and we will have the keywords with the highest weight on top, which then represents the most relevant keywords in the visit up until now, and acts as our prediction for what the user is looking for.
Instead of only looking at the user alone, we can also compare their information need to information needs of other users, and by looking at where the other users end up, we can predict where our user might want to end up.

Optionally, a decay factor can be introdced. Essentially, every iteration, all values are divided by a factor, meaning that keywords on recent pages are more biased and weighted more heavily.

It is also possible to give more weight to pages based on how much time the user spent on that page.

In [10]:
from pandas import DataFrame
#Method which essentially just takes the keywords and their weights on each page the user visits, continues to the next url, and then sums the weights.
#Be sure to realise that only passing a path will execute the normal implementation, which simply sums all new tf-idf values of each url.
#This is because the default decay_factor value is 1, meaning that nothing changes, and the default secs is None, if that is None, the if statement with secs is never entered and we do not take time into account.
#Sorted gives an option to sort the vector before returning, this puts the most relevant keywords on top, but does not allow for easy similarity comparison, thus should only be used if this is not the intention
def find_keyword_weights(path, decay_factor=1, secs=None, sorted=False):
    #initiate array of weights (weights start as 0, size=num of keywords)
    weights = np.zeros(tfidf.get_number_of_keywords())
    #ensures all output is visible
    pd.options.display.max_rows = 0
    #Iterate over each page url
    for i in range(len(path)):
        #Divide the weights by the decay factor (1 by default, no decay in that case)
        weights = weights/decay_factor
        #Initialize the tf-idf weights of the current page
        new_weights = tfidf.get_all_keywords_by_id_normalized(path[i])
        #If we are taking time into account, multiply the weights by the amount of time spent on this page
        if secs is not None:
            new_weights*=secs[i]
        #Add the new weights to the existing weights
        weights += new_weights
        #Normalize such that the largest value becomes 1
        if max(weights)==0:
            continue
        weights /= max(weights)
    #Option to sort s.t. max weights are on top.
    if sorted:
        df = DataFrame(weights, columns=['weights'])
        return df.sort_values(by=['weights'], ascending=False)
    return weights

Method which calculates keywords of a visit

In [11]:
def calculatePercentComplete(decimal_complete):
    return round(decimal_complete, 3)*100

In [12]:
def compute_keywords_of_complete_visit(visit_ids, paths):
    total_paths = len(paths)
    all_visits = {}
    keys = []
    keywords_weight_dict = {}
    keywords_weights_top_10 = {}
    for i in range(tfidf.get_number_of_keywords()):
        keys.append(tfidf.get_keyword_by_id(i))
    
    previous_percent = -1
    for index, path in enumerate(paths):
        weights = find_keyword_weights(path, decay_factor=28) #!!!! CAREFUL make sure you have the correct parameters
        keywords_weights_dict = dict(zip(keys, weights))
        series = pd.Series(keywords_weights_dict)
        series.sort_values(axis=0, ascending=False, inplace=True, kind='quicksort', na_position='last', ignore_index=False, key=None)
        keywords_weights_dict_sorted = series.to_dict()
        keywords_weights_top_10 = dict(zip(list(keywords_weights_dict_sorted.keys())[:10], list(keywords_weights_dict_sorted.values())[:10]))
        all_visits[visit_ids[index]] = keywords_weights_top_10
        percent = calculatePercentComplete(index/total_paths)
        if percent != previous_percent:
            print(f"{percent}% done...")
            previous_percent = percent

    print("DONE!")
    return all_visits

In [21]:
all_keywords_and_weights_with_visit_ids = compute_keywords_of_complete_visit(visit_ids, paths)

0.0% done...
0.1% done...
0.2% done...
0.3% done...
0.4% done...
0.5% done...
0.6% done...
0.7000000000000001% done...
0.8% done...
0.8999999999999999% done...
1.0% done...
1.0999999999999999% done...
1.2% done...
1.3% done...
1.4000000000000001% done...
1.5% done...
1.6% done...
1.7000000000000002% done...
1.7999999999999998% done...
1.9% done...
2.0% done...
2.1% done...
2.1999999999999997% done...
2.3% done...
2.4% done...
2.5% done...
2.6% done...
2.7% done...
2.8000000000000003% done...
2.9000000000000004% done...
3.0% done...
3.1% done...
3.2% done...
3.3000000000000003% done...
3.4000000000000004% done...
3.5000000000000004% done...
3.5999999999999996% done...
3.6999999999999997% done...
3.8% done...
3.9% done...
4.0% done...
4.1000000000000005% done...
4.2% done...
4.3% done...
4.3999999999999995% done...
4.5% done...
4.6% done...
4.7% done...
4.8% done...
4.9% done...
5.0% done...
5.1% done...
5.2% done...
5.3% done...
5.4% done...
5.5% done...
5.6000000000000005% done...
5.7%

In [22]:
with open('C:/Users/01din\Desktop\data APG\keywords_split/dino_keywords.json', 'w') as jf: #your_file_name_here!!!!!
    json.dump(all_keywords_and_weights_with_visit_ids, jf)